### BELOW CODE IS TO WORK WITH SPARK IN COLAB, IGNORE IF NOT USING COLAB

In [1]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,081 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,691 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restr

In [2]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark import SparkConf
os.environ['PYSPARK_PYTHON']=sys.executable
os.environ['PYSPARK_DRIVER_PYTHON']=sys.executable

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('week2').getOrCreate()
spark

In [4]:
dataframe=spark.createDataFrame([
    (1,'harika'),
    (2,'Mano'),
    (3,'Bhavya'),
    (4,'Vaibhav'),
    (5,'Aditi'),
    (6,'Aarushi'),
    (7,'Harry'),
    (8,'Karthik'),
    (9,'Aman'),
    (10,'sidpai'),
    (11,'Abhiram')
],['id','name'])

In [5]:
dataframe.show()

+---+-------+
| id|   name|
+---+-------+
|  1| harika|
|  2|   Mano|
|  3| Bhavya|
|  4|Vaibhav|
|  5|  Aditi|
|  6|Aarushi|
|  7|  Harry|
|  8|Karthik|
|  9|   Aman|
| 10| sidpai|
| 11|Abhiram|
+---+-------+



1) Implement a PySpark script that applies transformations like filter and withColumn on a
DataFrame.

In [6]:
namesWithA=dataframe.filter(dataframe["name"].startswith('A'))

In [7]:
namesWithA.show()

+---+-------+
| id|   name|
+---+-------+
|  5|  Aditi|
|  6|Aarushi|
|  9|   Aman|
| 11|Abhiram|
+---+-------+



In [8]:
# incrID=df.withColumn("new column name",df['existing column name'] operation)
incrID=dataframe.withColumn("incremented",dataframe['id']+2)
incrID.show()

+---+-------+-----------+
| id|   name|incremented|
+---+-------+-----------+
|  1| harika|          3|
|  2|   Mano|          4|
|  3| Bhavya|          5|
|  4|Vaibhav|          6|
|  5|  Aditi|          7|
|  6|Aarushi|          8|
|  7|  Harry|          9|
|  8|Karthik|         10|
|  9|   Aman|         11|
| 10| sidpai|         12|
| 11|Abhiram|         13|
+---+-------+-----------+



2) Write a PySpark script that performs actions like count and show on a DataFrame.

In [9]:
print(f"total rows:{dataframe.count()}")

total rows:11


In [10]:
print(f"total name with A:{dataframe.filter(dataframe['name'].startswith('A')).count()}")
print(f"total name with b:{dataframe.filter(dataframe['name'].startswith('B')).count()}")
print(f"total name with h:{dataframe.filter(dataframe['name'].startswith('h')).count()}")


total name with A:4
total name with b:1
total name with h:1


In [11]:
incrID.filter(incrID['incremented']>6).show()

+---+-------+-----------+
| id|   name|incremented|
+---+-------+-----------+
|  5|  Aditi|          7|
|  6|Aarushi|          8|
|  7|  Harry|          9|
|  8|Karthik|         10|
|  9|   Aman|         11|
| 10| sidpai|         12|
| 11|Abhiram|         13|
+---+-------+-----------+



3) Demonstrate how to perform basic aggregations (e.g., sum, average) on a PySpark
DataFrame.


In [12]:
for column, datatype in dataframe.dtypes:
    print(f"{column}: {datatype}")

id: bigint
name: string


In [13]:
dataframe = dataframe.withColumn("id", dataframe["id"].cast("int"))

In [14]:
for column, datatype in dataframe.dtypes:
    print(f"{column}: {datatype}")

id: int
name: string


In [15]:
from pyspark.sql import functions as f
dataframe.agg(f.avg(dataframe['id'])).show()
dataframe.agg(f.sum(dataframe['id'])).show()
dataframe.agg(f.max(dataframe['id'])).show()
dataframe.agg(f.min(dataframe['id'])).show()

+-------+
|avg(id)|
+-------+
|    6.0|
+-------+

+-------+
|sum(id)|
+-------+
|     66|
+-------+

+-------+
|max(id)|
+-------+
|     11|
+-------+

+-------+
|min(id)|
+-------+
|      1|
+-------+



4) Show how to write a PySpark DataFrame to a CSV file.

In [17]:
path='/content/writtenData.csv'
dataframe.write.csv(path,mode='append')

5) Implement wordcount program in PySpark.


In [36]:
sc=SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [37]:
path='/content/sentences.txt'
lines=sc.textFile(path)

split the lines into words and create one big list

In [38]:
words=lines.flatMap(lambda line: line.split(" "))
words.take(10)

['hello',
 'gello',
 'kello',
 'wello',
 'hello',
 'hello',
 'gello',
 'kello',
 'mano',
 '']

In [39]:
pairRDD=words.map(lambda word: (word,1))
pairRDD.take(10)

[('hello', 1),
 ('gello', 1),
 ('kello', 1),
 ('wello', 1),
 ('hello', 1),
 ('hello', 1),
 ('gello', 1),
 ('kello', 1),
 ('mano', 1),
 ('', 1)]

count the occurences of each unique word

In [40]:
counts=pairRDD.reduceByKey(lambda x,y:x+y)
sorted_counts=counts.sortBy(lambda x:x[1])
sorted_counts.collect()

[('', 1), ('wello', 1), ('mano', 1), ('gello', 2), ('kello', 2), ('hello', 3)]